In [ ]:
pip install lightgbm optuna

In [9]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier

pd.set_option('display.max_columns', None, 'display.max_rows', None)

In [10]:
## Reading the data
train = pd.read_csv('Data/W23P2_train.csv')
test = pd.read_csv('Data/W23P2_test.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

In [11]:
## Removing training observations with prices higher than $40,000
train = train[train['price'] < 40000].reset_index(drop = True)

## Creating columns for outlier locations
train['location_outlier'] = np.where(train['latitude'] < 35, 1, 0)
test['location_outlier'] = np.where(test['latitude'] < 35, 1, 0)

## Log transformation on the price variable
train['price'] = np.log(train['price'])
test['price'] = np.log(test['price'])

In [12]:
## Defining input and target variables
X = train.drop(columns = ['street_address', 'interest_level'])
Y = train['interest_level']

## Splitting the data
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 42)

In [13]:
def lgbm_objective(trial):
    
    ## Defining the LGB hyper-parameter grid
    LGB_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1500, step = 100),
                      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01),
                      'num_leaves': trial.suggest_int('num_leaves', 5, 100, step = 5),
                      'max_depth': trial.suggest_int('max_depth', 3, 15),
                      'subsample': trial.suggest_float('subsample', 0.7, 1, step = 0.01), 
                      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1, step = 0.01),
                      'random_state': trial.suggest_int('random_state', 1, 1000),
                      'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.5, step = 0.1),
                      'reg_lambda': trial.suggest_float('reg_lambda', 0, 0.5, step = 0.1), 
                      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 0, 300, step = 20),
                      'verbosity': -1
                     }
                     
    ## Building the LightGBM model
    model = LGBMClassifier(**LGB_param_grid, n_jobs = -1, objective = 'multiclass').fit(X_train, Y_train)
        
    ## Predicting on the test data-frame
    lgbm_md_preds = model.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    lgbm_md_error = log_loss(Y_validation, lgbm_md_preds)
    
    return lgbm_md_error

In [15]:
## Starting LightGBM
## ----
## Creating a study object and to optimize the home objective function
study_lgbm = optuna.create_study(direction = 'minimize')
study_lgbm.optimize(lgbm_objective, n_trials = 1000)

[I 2023-04-13 21:13:16,461] A new study created in memory with name: no-name-f33c5cde-51b9-44b4-abc7-712d9eff595d


[LightGBM] [Warning] min_data_in_leaf is set=280, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=280


[I 2023-04-13 21:13:19,297] Trial 0 finished with value: 0.9208697968722737 and parameters: {'n_estimators': 1000, 'learning_rate': 0.13, 'num_leaves': 45, 'max_depth': 11, 'subsample': 0.94, 'colsample_bytree': 0.96, 'random_state': 317, 'reg_alpha': 0.2, 'reg_lambda': 0.1, 'min_data_in_leaf': 280}. Best is trial 0 with value: 0.9208697968722737.


[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


[I 2023-04-13 21:13:21,297] Trial 1 finished with value: 0.8335393852878876 and parameters: {'n_estimators': 400, 'learning_rate': 0.02, 'num_leaves': 45, 'max_depth': 14, 'subsample': 0.9199999999999999, 'colsample_bytree': 0.7, 'random_state': 322, 'reg_alpha': 0.0, 'reg_lambda': 0.5, 'min_data_in_leaf': 80}. Best is trial 1 with value: 0.8335393852878876.


...

[I 2023-04-13 21:27:54,843] Trial 997 finished with value: 0.8345951342221058 and parameters: {'n_estimators': 400, 'learning_rate': 0.12, 'num_leaves': 80, 'max_depth': 4, 'subsample': 0.76, 'colsample_bytree': 0.73, 'random_state': 84, 'reg_alpha': 0.5, 'reg_lambda': 0.1, 'min_data_in_leaf': 20}. Best is trial 760 with value: 0.8207507514460652.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)


[LightGBM] [Warning] min_data_in_leaf is set=0, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=0


[I 2023-04-13 21:27:55,595] Trial 998 finished with value: 0.8279160565882109 and parameters: {'n_estimators': 300, 'learning_rate': 0.11, 'num_leaves': 10, 'max_depth': 3, 'subsample': 0.77, 'colsample_bytree': 0.74, 'random_state': 1, 'reg_alpha': 0.5, 'reg_lambda': 0.2, 'min_data_in_leaf': 0}. Best is trial 760 with value: 0.8207507514460652.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


[I 2023-04-13 21:27:56,207] Trial 999 finished with value: 0.8316424761457901 and parameters: {'n_estimators': 200, 'learning_rate': 0.14, 'num_leaves': 15, 'max_depth': 3, 'subsample': 0.7799999999999999, 'colsample_bytree': 0.7799999999999999, 'random_state': 20, 'reg_alpha': 0.5, 'reg_lambda': 0.2, 'min_data_in_leaf': 20}. Best is trial 760 with value: 0.8207507514460652.


In [16]:
## Printing best hyper-parameter set
print('\nLightGBM: \n', study_lgbm.best_trial.params)
print(study_lgbm.best_trial.value)


LightGBM: 
 {'n_estimators': 400, 'learning_rate': 0.11, 'num_leaves': 70, 'max_depth': 3, 'subsample': 0.7799999999999999, 'colsample_bytree': 0.83, 'random_state': 1, 'reg_alpha': 0.5, 'reg_lambda': 0.2, 'min_data_in_leaf': 20}
0.8207507514460652


In [26]:
X_train = train.drop(columns = ['street_address', 'interest_level'])
Y_train = train['interest_level']

X_test = test.drop(columns = ['street_address', 'ID'])

In [27]:
lgbm_md = LGBMClassifier(n_estimators = 400, learning_rate = 0.11, num_leaves = 70, max_depth = 3, subsample = 0.78, 
                         colsample_bytree = 0.83, random_state = 1, reg_alpha = 0.5, reg_lambda = 0.2, min_data_in_leaf = 20).fit(X_train, Y_train)

## Making predictions
train_preds = lgbm_md.predict_proba(X_train)
test_preds = lgbm_md.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


In [28]:
## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
print(f'Training Error: {train_error}')

Training Error: 0.7027425287478086


In [29]:
## Creating submission file
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/lgbm_apr13.csv', index = False)

In [30]:
sub['high'] = np.where(sub['high'] < 0.1, 0, sub['high'])
sub['medium'] = np.where(sub['medium'] < 0.1, 0, sub['medium'])
sub['low'] = np.where(sub['low'] < 0.1, 0, sub['low'])

sub['high'] = np.where(sub['high'] > 0.8, 1, sub['high'])
sub['medium'] = np.where(sub['medium'] > 0.8, 1, sub['medium'])
sub['low'] = np.where(sub['low'] > 0.8, 1, sub['low'])

In [31]:
sub.to_csv('Submissions/lgbm_apr13_rounded.csv', index = False)